# Experimentation

In [7]:
%pip install ipywidgets

   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   ---------------------------------------- 139.4/139.4 kB 8.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/214.4 kB ? eta -:--:--
   --------------------------------------- 214.4/214.4 kB 13.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------  2.3/2.3 MB 73.7 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 49.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Get Video Information

In [100]:
import datetime
import json
import math

import requests
from yt_dlp import YoutubeDL

def list_most_recent_videos(days: int, channel: str):
    today = datetime.datetime.today()
    diff = datetime.timedelta(days=-days)
    target = today + diff
    target = target.strftime("%Y%m%d")
    params = {
        "dateafter": target, 
        "extractor-args": {"youtubetab": {"approximate_date": ['']}},
        "progress": True,
        # "write-pages": True,
    }

    with YoutubeDL(params=params) as ydl:
        info = ydl.extract_info(channel, download=False)
        return info

In [86]:
today = datetime.datetime.today()
diff = datetime.timedelta(days=-7)
(today + diff).strftime("%Y%m%d")

'20240615'

In [101]:
test = list_most_recent_videos(2, 'https://www.youtube.com/@CivilNetTV/videos')

[youtube:tab] Extracting URL: https://www.youtube.com/@CivilNetTV/videos
[youtube:tab] @CivilNetTV/videos: Downloading webpage
[download] Downloading playlist: CIVILNET - Videos
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 1: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 2: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 3: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 4: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 5: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 6: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 7: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 8: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 9: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 10: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 11: Downloading API JSON
[youtube:tab] UCxPJyX0oQWmqZ5akbuN5DTg page 12: Downloading API JSON
[yo

KeyboardInterrupt: 

In [2]:
info = list_most_recent_videos(0, "https://www.youtube.com/@adumb_codes/videos")

[youtube:tab] Extracting URL: https://www.youtube.com/@adumb_codes/videos
[youtube:tab] @adumb_codes/videos: Downloading webpage
[download] Downloading playlist: adumb - Videos
[youtube:tab] Playlist adumb - Videos: Downloading 6 items of 6
[download] Downloading item 1 of 6
[youtube] Extracting URL: https://www.youtube.com/watch?v=JheGL6uSF-4
[youtube] JheGL6uSF-4: Downloading webpage
[youtube] JheGL6uSF-4: Downloading ios player API JSON
[youtube] JheGL6uSF-4: Downloading m3u8 information
[download] Downloading item 2 of 6
[youtube] Extracting URL: https://www.youtube.com/watch?v=vLOgzi5y9xE
[youtube] vLOgzi5y9xE: Downloading webpage
[youtube] vLOgzi5y9xE: Downloading ios player API JSON
[youtube] vLOgzi5y9xE: Downloading m3u8 information
[download] Downloading item 3 of 6
[youtube] Extracting URL: https://www.youtube.com/watch?v=mTPqIuEM-ug
[youtube] mTPqIuEM-ug: Downloading webpage
[youtube] mTPqIuEM-ug: Downloading ios player API JSON
[youtube] mTPqIuEM-ug: Downloading m3u8 inform

In [75]:
with open('temp.json', 'w') as file:
    file.write(json.dumps(YoutubeDL().sanitize_info(info), indent=4))

In [ ]:
for i in range(len(info['entries'])):
    print("Title:", info['entries'][i]['title'])
    print("Description:", info['entries'][i]['description'])
    print("Link to captions (en-orig):", info['entries'][i]['automatic_captions']['en-orig'][0]['url'])

## Summarize

In [3]:
response = requests.get(info['entries'][0]['automatic_captions']['en-orig'][0]['url'])
response.status_code

200

In [ ]:
response.json()

In [4]:
def get_text(captions: dict) -> str:
    text = []
    for event in captions['events']:
        if 'segs' in event:
            for segment in event['segs']:
                text.append(segment['utf8'])
    return ''.join(text)

In [5]:
len(get_text(response.json()).split())

3216

In [8]:
import torch
from transformers import AutoTokenizer, BartForConditionalGeneration, pipeline

assert torch.cuda.is_available()

model_name = "facebook/bart-large-cnn"
bart = BartForConditionalGeneration.from_pretrained(
    model_name, 
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2"
    # device_map='cuda',
    ignore_mismatched_sizes=True,
).to('cuda')
# bart = pipeline('summarization', model=model_name, device='cuda')

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
text = get_text(response.json())

c:\Users\trraa\Documents\repos\youtube-summarizer\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [59]:
def summarize(text_batches: list) -> str:
    if isinstance(text_batches, str):
        text_batches = [text_batches]
    inputs = tokenizer(text_batches, max_length=1024, return_tensors='pt', padding=True, truncation=True)
    inputs = inputs.to('cuda')
    summary_ids = bart.generate(inputs['input_ids'], num_beams=2, min_length=0, max_length=100)
    result = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return result

In [41]:
summarize([text])[0]

'The graph is the culmination of months of work thousands of lines of code and an absurd amount of computation time to create a visual representation of Wikipedia. Each circle represents one of the 6.3 million English Wikipedia articles and these are the nearly 200 million links between these articles that form the network of \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0Wikipedia. The idea behind finding these communities is to group similar articles together since in the theory of knowledge theory articles which are more closelylinked to each other should also have more similar content'

In [50]:
# Get total approximate token length of the whole text
whole_inputs = tokenizer([text], return_tensors='pt', truncation=False, add_special_tokens=False)
num_tokens = len(whole_inputs['input_ids'][0])
print(num_tokens)
math.ceil(num_tokens / 1024)

4007


4

In [38]:
print(len(text))
print(len(text) // 4)
print(len(text) // 4 * 4)  # Close...

17877
4469
17876


In [48]:
whole_inputs

{'input_ids': tensor([[    0, 12196,    47,  ..., 50118, 34464,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [53]:
tokenizer.decode(whole_inputs['input_ids'][0][1024:2048])

" it looks like this 46,000\nincoming links is a lot but it's still a\nrelatively small amount when compared to\nthe most reference articles on Wikipedia\nfor example over 100,000 articles linked\nto the article for World War I however\neven that is significantly less than the\n189,000 articles that link to the\narticle for World War II on a side note\nI find it interesting how the graphs for\nWorld War I and World War II are similar\nto each other showing how articles with\na similar type of content also share\nsimilar links despite the article for\nWorld War II being referenced nearly\n200,000 times it's only the fourth most\nlink to article on the English Wikipedia\nthe second most referenced article has\nover 240,000 links to it and it belongs\nto the article for association football\nor soccer the interesting thing about\nthis graph is that even though it has\nmore links most of the Links come from\nwithin the same Community for football\narticles with a lot of links originating\n

In [55]:
num_batches = math.ceil(num_tokens / 1024)
token_batch_size = 1024
char_batch_size = len(text) // num_batches
text_batches = []
for i in range(num_batches):
    batch = tokenizer.decode(whole_inputs['input_ids'][0][i * token_batch_size : (i + 1) * token_batch_size])
    text_batches.append(batch)
# text_batches = [text[i * char_batch_size : (i + 1) * char_batch_size] for i in range(4)]
text_batches

["what you are looking at is a visual\nrepresentation of Wikipedia a website\nthat I can almost guarantee you've used\nat some point in your life each circle\nrepresents one of the 6.3 million\nEnglish Wikipedia articles and these are\nthe nearly 200 million links between\nthese articles that form the network of\nWikipedia this graph is the culmination\nof months of work thousands of lines of\ncode and an absurd amount of computation\ntime and I know what you might be\nthinking what a complete waste of time\nbecause other than looking mildly\ninteresting we can't really extract any\nuseful information from this graph in\nits current state but I promise that if\nyou stay with me and watch this video\nnot only will you come to understand\nthis graph but you just might learn a\ncouple of interesting things along the\n[Music]\nway\nlet's start by understanding the\ndifferent colors of this graph each\ncolor represents different communities\nwhich were algorithmically determined a\ncommunit

In [61]:
batch_summaries = summarize(text_batches)
for summary in batch_summaries:
    print(summary)

final_summary = ' '.join(summarize('\n'.join(batch_summaries))[0].split())
print(final_summary)

The graph is the culmination of months of work thousands of lines of code and an absurd amount of computation time to create a visual representation of Wikipedia. Each circle represents one of the 6.3 million English Wikipedia articles and these are the nearly 200 million links between articles that form the network of                Wikipedia. The idea behindfinding these communities is to group similar articles together since in the theory of knowledge theory articles which are more closelylinked to each other should also have more similar content.
The most referenced article on Wikipedia isn't about a famous person or a historical event it's about a country. 38% of all articles on Wikipedia makereference to articles of countries and countries. The United States is referenced more than any other article onWikipedia linked to by nearly 280,000 other articles.
2,000 dead-end and orphaned articles exist on Wikipedia articles. These articles have no incoming or outgoing links. The number

In [70]:
def summarize_long_text(text: str, token_batch_size: int = 1024, return_partial_summaries=False) -> str:
    # Get total approximate token length of the whole text
    whole_inputs = tokenizer([text], return_tensors='pt', truncation=False, add_special_tokens=False)
    num_tokens = len(whole_inputs['input_ids'][0])
    num_batches = math.ceil(num_tokens / token_batch_size)

    # Batchify text
    text_batches = []
    for i in range(num_batches):
        batch = tokenizer.decode(whole_inputs['input_ids'][0][i * token_batch_size : (i + 1) * token_batch_size])
        text_batches.append(batch)

    # Generate summaries
    batch_summaries = summarize(text_batches)
    final_summary = ' '.join(summarize('\n'.join(batch_summaries))[0].split())

    if return_partial_summaries:
        return final_summary, batch_summaries
    return final_summary

### Trying it out

In [73]:
index = 3
print("Title:", info['entries'][index]['title'])
response = requests.get(info['entries'][index]['automatic_captions']['en-orig'][0]['url'])
assert response.ok, response.status_code
text = get_text(response.json())
result = summarize_long_text(text, return_partial_summaries=True)
print("Summary:", result[0])

Title: Using AI to Create the Perfect Keyboard
Summary: The QWERTY keyboard was invented nearly 150 years ago by Christopher Latham. The keyboard layout has remained unchanged ever since.


In [74]:
result[1]

['The QWERTY keyboard was invented nearly 150 years ago by Christopher Latham scholes. The keyboard layout has remained unchanged ever since. I simulated a keyboard layout that is more efficient by minimizing the total distance the fingers have to travel to type.',
 'The genetic algorithm is used to create the next generation of keyboards. It uses the most efficient keyboards from each generation to create a new layout. The keyboard should become increasingly efficient over time as it mimics natural selection.',
 'The AI uses a genetic algorithm to group the keys on a keyboard. The most commonly used keys are placed closer to the starting positions of the two fingers. The semicolon key is now in a closer position to the home Keys. The rst lne keyboard layout might be the best for typing Standard English.']

## Summarize Video
Putting it together...

In [ ]:
def summarize_video(url: str):
    pass